In [1]:
import os
import pydicom
import nibabel as nib
import numpy as np
from skimage.transform import resize

target_shape = (128, 128)
# Define the data directories
data_directory = "/kaggle/input/rsna-2023-abdominal-trauma-detection"
segmentations_directory = os.path.join(data_directory, "segmentations")
train_images_directory = os.path.join(data_directory, "train_images")

# Create a directory for saving the NIfTI images
output_directory = "/kaggle/working/nifti_images"
os.makedirs(output_directory, exist_ok=True)

# Function to load and resize DICOM images from a directory
def load_and_resize_dicom_images(dicom_dir, target_shape):
    dicom_files = [os.path.join(dicom_dir, filename) for filename in os.listdir(dicom_dir)]
    dicom_files.sort(key=lambda x: pydicom.dcmread(x).InstanceNumber)
    dicom_images = [pydicom.dcmread(file).pixel_array for file in dicom_files]
    dicom_images.reverse()  # Reverse the order of DICOM images
    resized_images = [resize(image, target_shape, preserve_range=True, anti_aliasing=False).astype(np.float32) for image in dicom_images]
    return resized_images

# List all segmentation files
segmentation_files = os.listdir(segmentations_directory)

# Define the range of folders to process (starting and ending indices)
starting_index = 800  # Change this to specify the starting index
ending_index = 1200  # Change this to specify the ending index

# Get a list of DICOM folders to process within the specified range
patient_folders_to_process = sorted(os.listdir(train_images_directory))[starting_index:ending_index]

# Iterate through each DICOM folder in the specified range
for patient_folder in patient_folders_to_process:
    if not os.path.isdir(os.path.join(train_images_directory, patient_folder)):
        continue  # Skip if not a directory
    for series_id in os.listdir(os.path.join(train_images_directory, patient_folder)):
        dicom_dir = os.path.join(train_images_directory, patient_folder, series_id)
        
        # Check if there is a matching segmentation file
        matching_segmentation_file = f"{series_id}.nii"
        if matching_segmentation_file in segmentation_files:
            continue  # Skip if there's a matching segmentation file
        
        # Load and resize DICOM images to (128, 128) as float32
        resized_dicom_images = load_and_resize_dicom_images(dicom_dir, target_shape)

        # Create a 3D NIfTI image with the shape of the mask
        nifti_image = np.stack(resized_dicom_images, axis=-1)
        nifti_image = nib.Nifti1Image(nifti_image, np.eye(4))

        # Save the NIfTI image with the same series_id in the output directory
        nifti_save_path = os.path.join(output_directory, f"{series_id}.nii")
        nib.save(nifti_image, nifti_save_path)

        print(f"Created NIfTI image for series_id: {series_id} and saved it to {nifti_save_path}")


Created NIfTI image for series_id: 11737 and saved it to /kaggle/working/nifti_images/11737.nii
Created NIfTI image for series_id: 38572 and saved it to /kaggle/working/nifti_images/38572.nii
Created NIfTI image for series_id: 40335 and saved it to /kaggle/working/nifti_images/40335.nii
Created NIfTI image for series_id: 5472 and saved it to /kaggle/working/nifti_images/5472.nii
Created NIfTI image for series_id: 52251 and saved it to /kaggle/working/nifti_images/52251.nii
Created NIfTI image for series_id: 47940 and saved it to /kaggle/working/nifti_images/47940.nii
Created NIfTI image for series_id: 51881 and saved it to /kaggle/working/nifti_images/51881.nii
Created NIfTI image for series_id: 22929 and saved it to /kaggle/working/nifti_images/22929.nii
Created NIfTI image for series_id: 50707 and saved it to /kaggle/working/nifti_images/50707.nii
Created NIfTI image for series_id: 55034 and saved it to /kaggle/working/nifti_images/55034.nii
Created NIfTI image for series_id: 32510 a

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt

# Define the path to the NII image
nii_path = '/kaggle/working/nifti_images/50172.nii'

# Load the NII image
nii_img = nib.load(nii_path)
nii_data = nii_img.get_fdata()

# Define the range of slices you want to view (from 0 to 60)
start_slice = 60
end_slice = 80

# Calculate the number of rows needed
num_slices = end_slice - start_slice
num_rows = (num_slices + 4) // 5  # 5 images per row

# Create subplots for the rows
fig, axs = plt.subplots(num_rows, 5, figsize=(15, 5 * num_rows))

# Plot the slices
for row in range(num_rows):
    for col in range(5):
        slice_num = start_slice + row * 5 + col
        if slice_num < end_slice:
            slice_data = nii_data[:, :, slice_num]  # Extract the 2D slice data
            axs[row, col].imshow(slice_data, cmap='gray')  # Display the slice
            axs[row, col].set_title(f'Slice {slice_num}')
            axs[row, col].axis('off')
 
plt.tight_layout()
plt.show()